**This notebook walks through the steps of applying transfer learning for image recognition.  Pytorch has many pretrained models that can be adapted to a given application.  The pretrained models simply need to be loaded and adapated for the given application.**

**For this case, ten objects will be classified from the CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html) data set.  The pretrained Inception-v3 (https://arxiv.org/abs/1512.00567) model will be adapted to classify these images.**

**The ten objects to be classified are:**

*   airplane
*   automobile
*   bird
*   cat
*   deer
*   dog
*   frog
*   horse
*   ship
*   truck

**Note that truck here refers to large semi type trucks.**

**First load the necessary packages.  Note that 'models' is being imported from 'torchvision'.  This gives access to many pretrained models.  Pretrained models can be obtained elsewhere as well but this package gives plenty of options for our purposes.**




In [77]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision
from torchvision import datasets, transforms, models

#from collections import OrderedDict

**The CIFAR-10 data set is included in the `torchvision.datasets` module.  It is loaded below.**

**Images need to undergo transformations before being loaded.  First the transforms for the training set are defined.**

**The random rotation aids in training.  It will allow the network to see the image from different angles during each training pass.  This will help the network to generalize.  Randomness is also added to the cropping step.  The inception v3 net is unique in that it expects size 299 (299x299 pixels).  Most models take size 224 but sometimes it is fun to be different.  A random horizontal flip is added to further aid in generalization and the normalization is defined to fit what the pretrained network expects.**

In [78]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(299),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


**No randomness is added to the test set as accuracy should be graded on the true images.  However, they are resized and then cropped to fit what the net expects.**

In [79]:
test_transforms = transforms.Compose([transforms.Resize(320),
                                      transforms.CenterCrop(299),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


**The train and test set are loaded below using the data loader utility.  Batches of 64 images at a time will be passed through the model.  The training set will be randomly shuffled to prevent the net from picking up patterns based on the order images are seen.  This is not necessary for the test set.  Notice that the train and test transforms are applied at this step.**  

In [80]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, drop_last = True)



Files already downloaded and verified
Files already downloaded and verified


**The classes are defined listed below in the order they are indexed (alphabetically).**

In [81]:
classes = ('airplane', 'automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

**The inception v3 model is loaded and described below.  It is pretrained but the output configuration needs to be determined in order to properly attach new layers.  Only 10 output layers are needed for this classification problem whereas the pretrained model outputs 1,000.  The key here is to recognize that the last layer takes 2,048 features and is named as (fc).  A small neural net will basically be attached to this layer and a new output layer will be constructed.  Syntactically this new small net will be named as (fc) and will replace the old output layer.**

In [82]:
model = models.inception_v3(pretrained = True)
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

**The first step in constructing a custom net from a pretrained net is to freeze the pretrained model.  New layers will be trained but the everything about the pretrained model being leveraged should remain the same.  Therefore the gradients are turned off.**

In [83]:
for param in model.parameters():
    param.requires_grad = False

**Now the old output layer will be replaced with the small net defined below.  It will start with a layer that takes 2,048 features as its input and it will output a log probability for each class.**

**Two fully connected rectified linear (ReLU) layers are built.  The first takes in the 2,048 features the model currently inputs to its output layer and instead outputs 500 features.  The next layer inputs those 500 and outputs 250 features.  Finally, those 250 features are converted into 10 outputs with a log softmax layer.  Note that dropout of .2 is used for both ReLU layers to combat overfitting.  This image classifier is trained to recognize 1,000 different image classes and this application only needs to classify 10**

**A good exercise would be to play with this architecture and see how it affects final accuracy.  Only a few tests were done on different configurations due to a limited gpu budget and time constraints.**

In [84]:
model.fc = nn.Sequential(nn.Linear(2048, 500),
                                 nn.ReLU(),
                                 nn.Dropout(.2), 
                                 nn.Linear(500, 250),
                                 nn.ReLU(),
                                 nn.Dropout(.2), 
                                 nn.Linear(250, 10),
                                 nn.LogSoftmax(dim=1))          

**Negative log likelihood loss is chosen as the cost function.  Adam is chosen as the optimizer.  Documentation on the Adam optimizer is easy to find but outside the scope of this tutorial.  At this point, it is sufficient to know that it is basically a "souped up" version of gradient descent.**

**Note the `to("cuda")` command when defining the loss function.  This is the first of a few objects that needs to be moved to the gpu to drastically speed up training.

In [85]:
criterion = nn.NLLLoss().to("cuda")
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

**Now the model is ready to be trained.  Loss is being tracked after every batch just for fun.**

**Much of the code below is explained in the 'income_model_binary_nn.ipynb' notebook in the same repo as this notebook.  This code is tracked differently simply for demonstration purposes and because it is fun to watch the model work (at least for me).**

**A big difference worth addressing is following two lines:**

                  `logps_t = model.forward(images)
                   logps = logps_t[0] #inception v3 outputs a tuple in train mode`
                    
**The code above is necessary because the inception v3 model outputs a tuple containing two tensors.  Position `[0]` in the tensor is the log proabilities that are needed to make predictions.  Position `[1]` contains the auxiliary logits with dimesionality equal to the original model's output (1,000).  That output is not needed so the log probabilities are stored and the auxiliary logits are ignored.**

**A fun exercise would be to store the batch and loss and make some plots that (hopefully) show the loss decreasing.  This is usually tracked across epochs but that takes a lot longer to run and gpu time can be limited for most people.**

In [86]:
model = model.to("cuda") #model is also moved to the gpu

In [87]:
epochs = 1
batch = 0

for epoch in range(epochs):
     
    running_loss = 0
    
    for images, labels in trainloader:
      
        #images and labels are moved to gpu as well
        images = images.to("cuda")
        labels = labels.to("cuda")
        
        batch += 1
        
        model.train()
        
        optimizer.zero_grad()
        
        logps_t = model.forward(images)
        logps = logps_t[0] #inception v3 outputs a tuple in train mode
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
#         print(f"Epoch {epoch+1} Batch {batch}.. "
#               f"Train loss: {train_loss:.3f}.. ")

    train_loss = running_loss/batch

    print(f"Epoch {epoch+1}.. "
          f"Train loss: {train_loss:.3f}.. ")
            

Epoch 1.. Train loss: 1.821.. 


**Since this model has only been run for one epoch, the full model can be saved and training resumed at a later time.  Code to load the model is provided below as well.**

**First the google drive will be mounted and then the model will be saved in a folder called 'models' in the google drive.  The mounting code only needs to be run once per session so comment it out after running if models will be saved/loaded multiple times.**

In [88]:
# from google.colab import drive
# drive.mount('/content/gdrive')

**The model has been named 'inception_v3_transfer' and stored on the drive.  It can be loaded with the code that is currently commented out.**


In [89]:
# model_name = 'inception_v3_transfer.pt'
# path = F'/content/gdrive/My Drive/models/{model_name}'
# torch.save(model, path)

# model = torch.load('/content/gdrive/My Drive/models/inception_v3_transfer.pt')


**Predictions are made below.  Loss and accuracy are both tracked but should not change by batch outside of some random variation.  The model has been switched to evaluation mode so that it will remain static.  In regular practice, accuracy for each batch would be stored and aggregated to get an overall accuracy metric.**

**Note that in evaluation mode, this model now only outputs the tensor of log probabilities.  This is because the auxiliary logits are only needed for training.  And since log probabilities are less useful, the `torch.exp()` function is used to convert the output to probabilities.**

**The predictions and labels are also being stored to do some fun analysis.  `preds` and `true` will store the predictions and true labels, respectively, as a list of tensors.  Data manipulations will be performed to make it possible to get some specific information about model performance.**

In [90]:
batch = 0

preds = []
true = []

with torch.no_grad():
    model.eval()
    for images, labels in testloader:
      
      images = images.to("cuda")
      labels = labels.to("cuda")
      
      batch += 1
      
      logps = model.forward(images)
      test_loss = criterion(logps, labels)

      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy = torch.mean(equals.type(torch.FloatTensor)).item()
      
      true.append([labels.view(*top_class.shape).view(64)])
      preds.append([top_class.view(64)])
    

      print(f"Batch {batch}.. "
            f"Test loss: {test_loss:.3f}.. "
            f"Accuracy: {accuracy}.. ")

Batch 1.. Test loss: 1.256.. Accuracy: 0.59375.. 
Batch 2.. Test loss: 1.352.. Accuracy: 0.53125.. 
Batch 3.. Test loss: 1.344.. Accuracy: 0.515625.. 
Batch 4.. Test loss: 1.253.. Accuracy: 0.53125.. 
Batch 5.. Test loss: 1.189.. Accuracy: 0.53125.. 
Batch 6.. Test loss: 1.202.. Accuracy: 0.59375.. 
Batch 7.. Test loss: 1.351.. Accuracy: 0.546875.. 
Batch 8.. Test loss: 1.191.. Accuracy: 0.5625.. 
Batch 9.. Test loss: 1.129.. Accuracy: 0.609375.. 
Batch 10.. Test loss: 1.151.. Accuracy: 0.59375.. 
Batch 11.. Test loss: 1.155.. Accuracy: 0.546875.. 
Batch 12.. Test loss: 1.185.. Accuracy: 0.65625.. 
Batch 13.. Test loss: 1.281.. Accuracy: 0.578125.. 
Batch 14.. Test loss: 1.290.. Accuracy: 0.484375.. 
Batch 15.. Test loss: 1.160.. Accuracy: 0.578125.. 
Batch 16.. Test loss: 0.983.. Accuracy: 0.71875.. 
Batch 17.. Test loss: 1.181.. Accuracy: 0.65625.. 
Batch 18.. Test loss: 1.328.. Accuracy: 0.546875.. 
Batch 19.. Test loss: 1.036.. Accuracy: 0.6875.. 
Batch 20.. Test loss: 1.221.. Accu

**First note that accuracy is roughly 55% across all batches.  This is not bad considering the minimal amount of work needed to get this running.  Even better when considering the minimal amount of optimization performed to get the best configuration and hyperparamters.  Keep in mind, a random guess would give 10% accuracy.  5x improvement over random is not bad for one weekend of work.**

**This is the power of transfer learning.  If somone else has already done the heavy lifting, there is no need to go through all that yourself.  It is time consuming and expensive to architect very deep neural networks.  If someone has already incurred that expense, nobody else has to start from scratch again.**

**With all that being said, overall accuracy does not tell the whole story.  Is the model better on some classes than others?  When the model misses, is there systematic bias?  This will be interesting to evaluate.**

**First the output will be converted from lists to one dimensional tensors.  Those tensors will then be converted to arrays so that a convenient built-in function can be used.  Here the data is renamed at each step to allow tracking of the different data objects created.  It is fine to overwrite the old data structures with the new ones if the user is confident.**

In [91]:
# initialize empty tensors and make tensor conversions

true_tensor = torch.empty(len(true), 64)
preds_tensor = torch.empty(len(true), 64)

for i in range(len(true)):
  true_tensor[i] = torch.stack(true[i])
  preds_tensor[i] = torch.stack(preds[i])

# convert the tensor to 1-d
true_1d = true_tensor.view(true_tensor.shape[0]*true_tensor.shape[1])
preds_1d = preds_tensor.view(preds_tensor.shape[0]*preds_tensor.shape[1])

# convert the 1-d tensors to 1-d numpy arrays
true_class = true_1d.numpy()
predicted_class = preds_1d.numpy()


**Now a confusion matrix will be constructed.  A function is defined that can convert the raw counts to either precision or recall or simply keep the raw counts as a default.  Filling the matrix with either precision or recall sometimes gives better context than raw counts.  Typically when raw counts are displayed, most people start scrambling to convert to those percentages anyway.  However, viewing the raw counts is very useful as well.**

**It should be mentioned, the author has used the terms precision and recall loosely here as those would only technically be the diagonals of the outputted matrices.  These terms are used for brevity and hopefully the reader takes away the idea behind displaying and evaluating these values.**

**That being said, the confustion matrix will serve two purposes.  First, instead of showing overall accuracy, this will give an idea of which classes the model is better at classifying.  This could lead to many adjustments both within the model itself or in how the model is used.  Second, it shows what is happening when the model "misses".  No model is perfect but some are useful.  Knowing the limitations and biases inherent in a model will make it far more useful.**

**The function is defined below.  Explanation is given within the comments of the function.**

In [92]:
def cm_metrics(matrix_of_confusion, classes, metric = 'raw'):
    """
    This function returns a confusion matrix with recall or precision 
    instead of raw counts.  Recall is the number of correct predictions 
    divided by the number of true examples for a given class.  Precision
    is the number of correct predictions divided by the number predicted
    for a given class.  This function assumes that true labels are 
    represented by the x-axis and predictions are represented by the y-axis.
    
    inputs:
        matrix_ of_confusion: a numpy array (output of 
        sklearn.metrics.confusion_matrix()) and outputs a 

        classes: a list of class labels in the order they appear in 
        the confusion matrix (order of indexes)
        
        metric: either 'precision' or 'recall'
              
    returns: a pandas dataframe with the requested matrix
    """
    
    # ensure pandas is imported
    
    import pandas as pd
    
    # compute the correct matrix given the desired output
    
    if metric == 'recall':
        row_sums = matrix_of_confusion.sum(axis = 1)
        return_matrix = matrix_of_confusion/row_sums[:, None] 
    
    elif metric == 'precision':
        col_sums = matrix_of_confusion.sum(axis = 0)
        return_matrix = matrix_of_confusion/col_sums[None, :] 
        
    elif metric == 'raw':
        return_matrix = matrix_of_confusion
    
    #convert output to dataframe with classes displayed
    
    cmdf = pd.DataFrame(return_matrix)
    cmdf.columns = classes
    
    cl = pd.DataFrame(classes)
    cl.columns = ['Classes']
    
    return_df = pd.concat([cl, cmdf], axis = 1)
    
    return return_df
        

**The tuple of classes needs to be converted to a list and the builtin confusion matrix function run and stored.  Then the function is run on recall and precision in that order.  A perfect model would have all 1s on the diagonal.  The rest of the entries will require careful observation to understand how the model misses.**



In [93]:
# convert the classes tuple to a list
cl = list(classes)

# run built in function and store confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_class, predicted_class)


In [94]:
cm_metrics(cm, cl)


,Classes,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,airplane,591,33,37,5,10,2,12,3,270,35
1,automobile,10,826,4,11,1,0,13,4,61,69
2,bird,131,26,413,72,54,27,192,14,44,26
3,cat,24,35,53,326,11,142,247,25,39,95
4,deer,20,6,68,34,389,13,341,80,32,17
5,dog,14,21,46,159,11,507,95,55,37,52
6,frog,6,12,36,29,5,2,887,3,8,12
7,horse,32,16,27,40,96,52,72,529,37,96
8,ship,41,42,10,5,2,3,16,2,841,35
9,truck,22,223,2,12,3,4,14,2,85,633


In [95]:
cm_metrics(cm, cl, metric = 'recall')



,Classes,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,airplane,0.592184,0.033066,0.037074,0.005010,0.010020,0.002004,0.012024,0.003006,0.270541,0.035070
1,automobile,0.010010,0.826827,0.004004,0.011011,0.001001,0.000000,0.013013,0.004004,0.061061,0.069069
2,bird,0.131131,0.026026,0.413413,0.072072,0.054054,0.027027,0.192192,0.014014,0.044044,0.026026
3,cat,0.024072,0.035105,0.053159,0.326981,0.011033,0.142427,0.247743,0.025075,0.039117,0.095286
4,deer,0.020000,0.006000,0.068000,0.034000,0.389000,0.013000,0.341000,0.080000,0.032000,0.017000
5,dog,0.014042,0.021063,0.046138,0.159478,0.011033,0.508526,0.095286,0.055165,0.037111,0.052156
6,frog,0.006000,0.012000,0.036000,0.029000,0.005000,0.002000,0.887000,0.003000,0.008000,0.012000
7,horse,0.032096,0.016048,0.027081,0.040120,0.096289,0.052156,0.072217,0.530592,0.037111,0.096289
8,ship,0.041123,0.042126,0.010030,0.005015,0.002006,0.003009,0.016048,0.002006,0.843531,0.035105
9,truck,0.022000,0.223000,0.002000,0.012000,0.003000,0.004000,0.014000,0.002000,0.085000,0.633000


In [96]:
cm_metrics(cm, cl, metric = 'precision')


,Classes,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,airplane,0.663300,0.026613,0.053161,0.007215,0.017182,0.002660,0.006353,0.004184,0.185695,0.032710
1,automobile,0.011223,0.666129,0.005747,0.015873,0.001718,0.000000,0.006882,0.005579,0.041953,0.064486
2,bird,0.147026,0.020968,0.593391,0.103896,0.092784,0.035904,0.101641,0.019526,0.030261,0.024299
3,cat,0.026936,0.028226,0.076149,0.470418,0.018900,0.188830,0.130757,0.034868,0.026823,0.088785
4,deer,0.022447,0.004839,0.097701,0.049062,0.668385,0.017287,0.180519,0.111576,0.022008,0.015888
5,dog,0.015713,0.016935,0.066092,0.229437,0.018900,0.674202,0.050291,0.076709,0.025447,0.048598
6,frog,0.006734,0.009677,0.051724,0.041847,0.008591,0.002660,0.469561,0.004184,0.005502,0.011215
7,horse,0.035915,0.012903,0.038793,0.057720,0.164948,0.069149,0.038115,0.737796,0.025447,0.089720
8,ship,0.046016,0.033871,0.014368,0.007215,0.003436,0.003989,0.008470,0.002789,0.578404,0.032710
9,truck,0.024691,0.179839,0.002874,0.017316,0.005155,0.005319,0.007411,0.002789,0.058459,0.591589


**Knowing the strengths and weaknesses of a model is very important for two main reasons:**

**The first reason is that the model can then be used effectively in its current state.  Assigning a degree of confidence to different types of predictions can help to guide decisions made using the model's output.  It may cause someone to ignore certain types of predictions and/or take swift action based on others.  Ignorantly regarding every type of output as the same is a big mistake often made by relatively smart people.  Take the time to learn nuances of any model.**

**The other reason it is important is because knowledge of the current state can help accelerate development and get the model to a better state.  This could mean changing the inputs, architecture, hyperparameters, etc. of the model and retraining.  In this case, simply training longer would probably yield benefits.  It can also mean treating the probabilities differently.  In the above example automobile is predicted often when the model should be predicting truck.  Perhaps try shifting some of the automobile predictions with high truck probability to a truck assignment and assess accuracy.  There is no law stating that the highest probability gets the prediction.  It is usually the best method.  But there are exceptions.**

**What other trends in misclassification are present?  Are there any other possible solutions?**